In [6]:
from pathlib import Path

import altair as alt

from solar import read_solar_array_data, find_minimum_system_cost

alt.data_transformers.enable("vegafusion")

folder = Path('./texas')
actual = [fp for fp in folder.iterdir() if fp.stem.startswith('Actual')]
len(actual)

56

In [8]:
from tqdm import tqdm

result = []
my_iterator = [(fp, i) for fp in actual[:10] for i in range(40)]
for fp, i in tqdm(my_iterator):
    sol = read_solar_array_data(fp)
    result.append(
        (fp.stem, *find_minimum_system_cost(200e3, 200e3, 10e3*10**(0.1*i), sol['power'].tolist()))
    )
result

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [08:28<00:00,  1.27s/it]


[('Actual_33.55_-101.85_2006_DPV_37MW_5_Min',
  (200000.0, 200000.0, 10000.0),
  (1.293384020638886, 0.0, 1),
  (258676.8041277772, 0.0, 10000.0),
  (258676.8041277772, 268676.8041277772, 1147144.8073320698),
  (0.0, 0.7731655749898899, 0.0, 0.2342135033088303)),
 ('Actual_33.55_-101.85_2006_DPV_37MW_5_Min',
  (200000.0, 200000.0, 12589.254117941673),
  (1.311458962812504, 0.0, 1),
  (262291.7925625008, 0.0, 12589.254117941673),
  (262291.7925625008, 274881.04668044247, 1158015.4915899278),
  (0.0, 0.7625095625222148, 0.0, 0.23737251243766808)),
 ('Actual_33.55_-101.85_2006_DPV_37MW_5_Min',
  (200000.0, 200000.0, 15848.931924611135),
  (1.3273666230457772, 0.0, 1),
  (265473.3246091555, 0.0, 15848.931924611135),
  (265473.3246091555, 281322.25653376663, 1171558.2432030463),
  (0.0, 0.7533713614897131, 0.0, 0.24012656491121614)),
 ('Actual_33.55_-101.85_2006_DPV_37MW_5_Min',
  (200000.0, 200000.0, 19952.623149688796),
  (1.3421899622102598, 0.0, 1),
  (268437.99244205194, 0.0, 19952.623

In [14]:
labels = [
    'filename',
    'solar cost ($/MW)', 'battery cost ($/MW)', 'load cost ($/MW)',
    'array size (MW)', 'battery size (MWh)', 'load size (1 MW by definition)',
    'array cost ($)', 'battery cost ($)', 'load cost ($, normalized to 1 MW)',
    'total power system cost ($)', 'total system cost ($)', 'total cost per utilization ($)',
    'battery size relative to 1 MW array', 'load size relative to 1 MW array', 'annual battery utilization', 'annual load utilization'
]

results = pd.DataFrame([dict(zip(labels, [fname] + [x for tup in r for x in tup])) for fname, *r in result])
results.head()

,filename,solar cost ($/MW),battery cost ($/MW),load cost ($/MW),array size (MW),battery size (MWh),load size (1 MW by definition),array cost ($),battery cost ($),"load cost ($, normalized to 1 MW)",total power system cost ($),total system cost ($),total cost per utilization ($),battery size relative to 1 MW array,load size relative to 1 MW array,annual battery utilization,annual load utilization
0,Actual_33.55_-101.85_2006_DPV_37MW_5_Min,200000.0,200000.0,10000.000000,1.293384,0.0,1,258676.804128,0.0,10000.000000,258676.804128,268676.804128,1.147145e+06,0.0,0.773166,0.0,0.234214
1,Actual_33.55_-101.85_2006_DPV_37MW_5_Min,200000.0,200000.0,12589.254118,1.311459,0.0,1,262291.792563,0.0,12589.254118,262291.792563,274881.046680,1.158015e+06,0.0,0.762510,0.0,0.237373
2,Actual_33.55_-101.85_2006_DPV_37MW_5_Min,200000.0,200000.0,15848.931925,1.327367,0.0,1,265473.324609,0.0,15848.931925,265473.324609,281322.256534,1.171558e+06,0.0,0.753371,0.0,0.240127
3,Actual_33.55_-101.85_2006_DPV_37MW_5_Min,200000.0,200000.0,19952.623150,1.342190,0.0,1,268437.992442,0.0,19952.623150,268437.992442,288390.615592,1.188424e+06,0.0,0.745051,0.0,0.242666
4,Actual_33.55_-101.85_2006_DPV_37MW_5_Min,200000.0,200000.0,25118.864315,1.358161,0.0,1,271632.249368,0.0,25118.864315,271632.249368,296751.113683,1.209426e+06,0.0,0.736290,0.0,0.245365


In [16]:
alt.Chart(results).mark_line().encode(
    x=alt.X('load cost ($/MW)', scale=alt.Scale(type='log')),
    y='annual load utilization',
    color='filename',
    tooltip='filename',
).properties(width=800, height=600)

alt.Chart(...)

There seem to be maybe 3 categories of solar locations? Ones with low utilization in low load cost, and then bumping up to "normal" levels in high load cost scenarios. The ones that start pretty high regardless, and then the ones that stay low-ish. What makes a load low-utilization friendly? Very volatile solar production?

In [ ]:
exmaple of low: Actual 33.55